In [6]:
## we have 10,000 combinations of only 200 unique names and we are storing all of them! 
## what the flyweight design pattern does is store reference/index in a list of the 200 names! 
## and hence saving huge chunks of memory! 

import random
import string
import sys

class User:
    def __init__(self, name):
        self.name = name

class User2:
    strings = []

    def __init__(self, full_name):
        def get_or_add(s):
            if s in self.strings:
                return self.strings.index(s)
            else:
                self.strings.append(s)
                return len(self.strings)-1
        self.names = [get_or_add(x) for x in full_name.split(' ')]

    def __str__(self):
        return ' '.join([self.strings[x] for x in self.names])

def random_string():
    chars = string.ascii_lowercase
    return ''.join([random.choice(chars) for x in range(8)])


if __name__ == '__main__':
    users = []

    first_names = [random_string() for x in range(100)]
    last_names = [random_string() for x in range(100)]

    for first in first_names:
        for last in last_names:
            users.append(User(f'{first} {last}'))

    u2 = User2('Jim Jones')
    u3 = User2('Frank Jones')
    
    print(u2.names)
    print(u3.names)
    print(User2.strings)

    users2 = []

    for first in first_names:
        for last in last_names:
            users2.append(User2(f'{first} {last}'))


[0, 1]
[2, 1]
['Jim', 'Jones', 'Frank']


In [4]:
users[0].name

'fzdwcbxv izequves'

In [10]:
users2[0].names

[3, 4]

In [17]:
print(users2[0])

ytrfpral rqjhvsxx


In [18]:
users2[0].strings[3:5] 

['ytrfpral', 'rqjhvsxx']

In [14]:
len(User2.strings)

203

In [15]:
User2.strings[0:5]

['Jim', 'Jones', 'Frank', 'ytrfpral', 'rqjhvsxx']

In [23]:
########### Text Formatting 
## This is the antipattern where we are storing a huge list of booleans to indicate
## whether or not the corresponding character is upper or lower case

class FormattedText:
    def __init__(self, plain_text):
        self.plain_text = plain_text
        self.caps = [False] * len(plain_text)

    def capitalize(self, start, end):
        for i in range(start, end):
            self.caps[i] = True

    def __str__(self):
        result = []
        for i in range(len(self.plain_text)):
            c = self.plain_text[i]
            result.append(c.upper() if self.caps[i] else c)
        return ''.join(result)

## Flyweight pattern 

class BetterFormattedText:
    def __init__(self, plain_text):
        self.plain_text = plain_text
        ### Formatting will be a collection of range objects that may have different
        ## formatting options 
        self.formatting = []

    class TextRange:
        def __init__(self, start, end, capitalize=False, bold=False, italic=False):
            self.end = end
            self.bold = bold
            self.capitalize = capitalize
            self.italic = italic
            self.start = start

        def covers(self, position):
            return self.start <= position <= self.end

    def get_range(self, start, end):
        range = self.TextRange(start, end)
        self.formatting.append(range)
        return range

    def __str__(self):
        result = []
        for i in range(len(self.plain_text)):
            c = self.plain_text[i]
            for r in self.formatting:
                if r.covers(i) and r.capitalize:
                    c = c.upper()
            result.append(c)
        return ''.join(result)


if __name__ == '__main__':
    ft = FormattedText('This is a brave new world')
    ft.capitalize(10, 15)
    print(ft)

    bft = BetterFormattedText('This is a brave new world')
    bft.get_range(16, 19).capitalize = True
    print(bft)
    
    

This is a BRAVE new world
This is a brave NEW world


In [22]:
## TextRange is like a class attribute except that it is a class itself! 
##c=BetterFormattedText.TextRange(13,4)

In [30]:
import unittest

### provide an interface to capitalize a particular word in a sentence 
class Sentence:
    def __init__(self, plain_text):
        self.words = plain_text.split(' ')
        self.tokens = {}
    ## Sentence[i] will return a token class which could be capitalized
    ## indicating that word number i in the sentence should be capitalized 
    def __getitem__(self, item):
        wt = self.WordToken()
        self.tokens[item] = wt
        return self.tokens[item]

    class WordToken:
        def __init__(self, capitalize=False):
            self.capitalize = capitalize

    def __str__(self):
        result = []
        for i in range(len(self.words)):
            w = self.words[i]
            if i in self.tokens and self.tokens[i].capitalize:
                w = w.upper()
            result.append(w)
        return ' '.join(result)


class Evaluate(unittest.TestCase):
    def test_exercise(self):
        s = Sentence('alpha beta gamma')
        s[1].capitalize = True
        self.assertEqual(str(s), 'alpha BETA gamma')

In [26]:
 s = Sentence('alpha beta gamma')

In [29]:
s.tokens

{1: <__main__.Sentence.WordToken at 0x10c91c4a8>,
 2: <__main__.Sentence.WordToken at 0x10c91c5c0>}